# **Instituto Tecnológico Vale**

## **Introdução a Visão Computacional**

### **Aula 06 - Detecção de objetos com YOLO**

# YOLOv4 - Como fazer o treinamento do detector personalizado

Para realizar o treinamento é necessário que você primeiro tenha coletado as imagens do conjunto de treinamento e criado o seu dataset

Só lembrando que o conjunto de imagens para fazer o treinamento do detector customizado do YOLO deve conter:

* Imagens dos objetos que queremos reconhecer, já catalogadas (com os arquivos annotation)
* Arquivos obj.data e obj.names 
* Arquivo .cfg customizado
* Arquivo train.txt (test.txt é opcional)

Se estiver com todos esses arquivos tudo ok então pode continuar executando as etapas desse Colab.

## Preparar arquivos de treino


In [ ]:
# Preparar os arquivos de treino (Os arquivos para a prática estão salvos no dropbox)

# Arquivo obj.data com as informações dos dados
!wget -O obj.data https://www.dropbox.com/s/d6vhniaee2fhe2s/obj.data?dl=0

# Arquivo obj.names com os nomes das classes
!wget -O obj.names https://www.dropbox.com/s/65040j7rb53kdng/obj.names?dl=0

# Arquivo obj.zip com as imagens e labels de treino
!wget -O obj.zip https://www.dropbox.com/s/mn2f5haqeczvvlb/obj.zip?dl=0

# Arquivo obj.zip com as imagens e labels de validacao
!wget -O valid.zip https://www.dropbox.com/s/81ooyqz0xgndqjl/valid.zip?dl=0

# Arquivo com o cfg configurado para duas classes!
!wget -O yolov4_custom.cfg https://www.dropbox.com/s/y5pdf9mxyj4f7k1/yolov4_custom.cfg?dl=0

# Arquivo train.txt
!wget -O train.txt https://www.dropbox.com/s/666utvs1c02izcw/train.txt?dl=0

# Arquivo test.txt
!wget -O test.txt https://www.dropbox.com/s/m2347regiydw67e/test.txt?dl=0

### **Lembre-se que os dados carregados acima são para nossa prática.**

### Caso você queira treinar o seu próprio detector de objetos organize seus dados com a mesma estrutura e carregue-os nas pastas corretas conforme exemplo.

- Você pode subir os dados para o colab como desejar. A opção do dropbox para a prática foi por praticidade apenas.

### Lembre-se: é preciso muito mais imagem do que a nossa prática!

## Preparar ambiente

In [ ]:
# Verificar GPU

import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

In [ ]:
# Dowload Darknet

!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet

In [ ]:
ls

In [ ]:
# Preparar GPU e Compilar a biblioteca

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

!make

In [ ]:
# Baixar pesos do yolo pretreinados

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
ls

In [ ]:
# Testar detector 

!./darknet detect cfg/yolov4.cfg yolov4.weights data/person.jpg

In [ ]:
#Verificar resultado

import cv2
import matplotlib.pyplot as plt

def mostrar(caminho):
  img = cv2.imread(caminho)
  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
mostrar("predictions.jpg")

In [ ]:
#Visualizr o caminho atual que estamos
#Tem q ser a pasta darknet!

!pwd

### Extraindo nossos arquivos baixados para dentro das pastas corretas do darknet

In [ ]:
# Extrair nossos arquivos obj baixados na pasta do darknet

!unzip /content/obj.zip -d ./data/obj/

In [ ]:
# Extrair nossos arquivos de validaçao na pasta do darknet

!unzip /content/valid.zip -d ./data/valid/

In [ ]:
# Copiando arquivos de configuração

!cp /content/yolov4_custom.cfg ./cfg
!cp /content/obj.names ./data
!cp /content/obj.data ./data
!cp /content/train.txt ./data
!cp /content/test.txt ./data

## Preparar para treinar

In [ ]:
# Baixar pesos da camadas convolucionais

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
ls

In [ ]:
# Treinar!!

!./darknet detector train data/obj.data cfg/yolov4_custom.cfg yolov4.conv.137 -dont_show -map

## Continuando o treinamento

O darknet salva os pesos do seu treino de 1000 em 1000 épocas e quano encontra o melhor resultado. Estes arquivos são salvo na pasta que é indicada para backup no arquivo obj.data

- Caso você pare o treino em algum momento antes de terminar o comando abaixo inicia o treino com as escolha dos pesos dos últimos melhores pesos, ou seja, não precisa começar do zero.

In [ ]:
# Só chamar o train com os últimos pessos salvos

!./darknet detector train data/obj.data cfg/yolov4_custom.cfg /backup/yolov4_custom_last.weights -dont_show -map

## Verificando o mAP (Mean Average Precision) do modelo 

In [ ]:
# Chamar detctor map e escolher modelo salvo desejado

!./darknet detector map data/obj.data cfg/yolov4_custom.cfg /backup/yolov4_custom_best.weights

## OBS:

### - O Google colab é uma plataforma compartilhada que te dá acesso contínuo por 12h. Algumas vezes o ambiente pode estar um pouco lento para rodar a GPU.

### - Se você deixar rodando um treinamento e ficar muito tempo sem acessar o ambiente, ele pode ser interrompido.

### - O google monitora seu uso de GPU e se você usar demais em um período curto de tempo ele te dá um block temporário no sistema! É sério! :)

### - Já está disponível no Brasil o google colab pró. Um colab pró com acesso sem interrupção por 24h e algumas outras diferenças.
